# exp070

In [1]:
import os
import sys
import traceback
import gc
import time
import random
import pickle
import pathlib
import subprocess
from dataclasses import dataclass
from collections import defaultdict

import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.model_selection import GroupKFold
import lightgbm as lgb
import optuna.integration.lightgbm as optuna_lgb

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import itertools

import warnings
warnings.simplefilter('ignore')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
@dataclass
class Cfg:
    mode = "local_cv" # "local_cv" or "kaggle_inf" 
    exp_name = "exp070_hyp_opt"
    input_dir = "/mnt/predict-student-performance-from-game-play/input/"
    output_dir = "/mnt/predict-student-performance-from-game-play/output/"
    prep_dir = "/mnt/predict-student-performance-from-game-play/prep/"
    seed = 42
    n_splits = 5
    best_threshold = 0.630 # local_cvの結果を入れる
cfg = Cfg()

if cfg.mode == "local_cv":
    os.makedirs(os.path.join(cfg.output_dir, cfg.exp_name), exist_ok=True)
    os.makedirs(os.path.join(cfg.output_dir, cfg.exp_name, "cache"), exist_ok=True)

elif cfg.mode == "kaggle_inf":
    import jo_wilder_310

In [3]:
params = {
    'objective': 'binary', 
    'boosting': 'gbdt', 
    'learning_rate': 0.1, 
    'metric': 'binary_logloss', 
    'seed': cfg.seed
} 

In [4]:
level_group_list = ['0-4', '5-12', '13-22']
level_group_map = {
    "q1":"0-4", "q2":"0-4", "q3":"0-4",
    "q4":"5-12", "q5":"5-12", "q6":"5-12", "q7":"5-12", "q8":"5-12", "q9":"5-12", "q10":"5-12", "q11":"5-12", "q12":"5-12", "q13":"5-12",
    "q14":"13-22", "q15":"13-22", "q16":"13-22", "q17":"13-22", "q18":"13-22"  
}

In [5]:
with open(cfg.prep_dir + 'cat_col_lists.pkl', 'rb') as f:
    cat_col_lists = pickle.load(f) 

In [6]:
def transform_labels_df_train(labels_):
    """
    labelsデータを整形する
    """
    labels = labels_.copy()
    labels["question"] = labels["session_id"].apply(lambda x: x.split("_")[1].replace("q", "")).astype(int)
    labels["session_id"] = labels["session_id"].apply(lambda x: x.split("_")[0]).astype(int)

    # trainの特徴量と結合するためにquestionに対応するlabel_groupを列として設けておく
    labels["level_group"] = ""
    labels.loc[labels["question"]<=3, "level_group"] = "0-4"
    labels.loc[(labels["question"]>=4)&(labels["question"]<=13), "level_group"] = "5-12"
    labels.loc[labels["question"]>=14, "level_group"] = "13-22"

    return labels


def transform_labels_df_inf(labels_):
    """
    labelsデータを整形する
    """
    labels = labels_.copy()
    labels["question"] = labels["session_id"].apply(lambda x: x.split("_")[1].replace("q", "")).astype(int)
    labels["session_id"] = labels["session_id"].apply(lambda x: x.split("_")[0]).astype(int)

    return labels

In [7]:
class FeaturesTrain:
    def __init__(self, sessions_df, labels):
        self.sessions_df = sessions_df.sort_values(["session_id", "level_group", "elapsed_time"], ignore_index=True)
        self.features = self.sessions_df[["session_id", "level_group"]].drop_duplicates().copy()
        self.result = labels
        self.group = sessions_df["level_group"].values[0]

    def _prep(self):
        self.sessions_df["time_diff"] = self.sessions_df["elapsed_time"] - self.sessions_df.groupby(["session_id", "level_group"])["elapsed_time"].shift(1)

    def _total_record_cnt(self):
        """level_groupごとのレコード数
        """
        add_features = self.sessions_df.groupby(["session_id", "level_group"])["index"].count().reset_index().rename(columns={"index":f"{self.group}_record_cnt"})
        self.features = self.features.merge(add_features, on=["session_id", "level_group"], how="left")

    def _group_elapsed_time(self):
        """level_groupごと、epapsed_timeのmax - min（経過時間）
        """
        add_features = self.sessions_df.groupby(["session_id", "level_group"])["elapsed_time"].agg([max,min]).reset_index()
        add_features[f"{self.group}_group_elapsed_time"] = add_features["max"] - add_features["min"]
        add_features[f"{self.group}_group_elapsed_time"] = add_features[f"{self.group}_group_elapsed_time"].astype(np.float32)
        add_features = add_features[["session_id", "level_group", f"{self.group}_group_elapsed_time"]].copy()
        self.features = self.features.merge(add_features, on=["session_id", "level_group"], how="left")

    def _cat_record_cnt(self, cat_col):
        """level_groupごと、各{cat}のレコード数
        """
        cat_list = cat_col_lists[self.group][cat_col]
        add_features = self.sessions_df.groupby(["session_id", "level_group", cat_col])["index"].count().reset_index().rename(columns={"index":"cnt"})
        for cat in cat_list:
            feat_name = f"{self.group}_{cat_col}_{str(cat)}_record_cnt"
            tmp = add_features[add_features[cat_col]==cat][["session_id", "level_group", "cnt"]].copy()
            if len(tmp) > 0:
                tmp = tmp.rename(columns={"cnt": feat_name})
                self.features = self.features.merge(tmp, on=["session_id", "level_group"], how="left")
                self.features[feat_name] = self.features[feat_name].fillna(0)
            else:
                self.features[feat_name] = 0

    def _cat_col_nunique(self, cat_col):
        """level_groupごと、[col]のユニーク数
        """
        add_features = self.sessions_df.dropna(subset=[cat_col]).drop_duplicates(["session_id", "level_group", cat_col])
        add_features = add_features.groupby(["session_id", "level_group"])["index"].count().reset_index().rename(columns={"index":f"{self.group}_{cat_col}_nunique"})
        self.features = self.features.merge(add_features, on=["session_id", "level_group"], how="left")        

    def _agg_features(self, val_cols, aggs):
        new_cols = [f"{self.group}_{v}_{a}" for v,a in itertools.product(val_cols, aggs)]
        add_features = self.sessions_df.groupby(["session_id", "level_group"])[val_cols].agg(aggs).reset_index()
        add_features.columns = ["session_id", "level_group"] + new_cols
        add_features[new_cols] = add_features[new_cols].astype(np.float32)
        self.features = self.features.merge(add_features, on=["session_id", "level_group"], how="left")

    def _cat_agg_features(self, val_cols, aggs, cat_col, not_use_cats=None):
        add_features = self.sessions_df.groupby(["session_id", "level_group", cat_col])[val_cols].agg(aggs).reset_index()

        if not_use_cats is not None:
            cat_list = [c for c in cat_col_lists[self.group][cat_col] if c not in not_use_cats]
        else:
            cat_list = cat_col_lists[self.group][cat_col]

        for cat in cat_list:
            new_cols = [f"{self.group}_{cat_col}_{cat}_{v}_{a}" for v,a in itertools.product(val_cols, aggs)]
            tmp = add_features[add_features[cat_col]==cat].copy()
            if len(tmp) > 0:
                tmp.columns = ["session_id", "level_group", cat_col] + new_cols
                tmp = tmp.drop(columns=[cat_col])
                self.features = self.features.merge(tmp, on=["session_id", "level_group"], how="left")
                self.features[new_cols] = self.features[new_cols].fillna(-1)
            else:
                self.features[new_cols] = -1
            self.features[new_cols] = self.features[new_cols].astype(np.float32)

    def get_train(self):
        self._prep()
        self._total_record_cnt()
        self._group_elapsed_time()
        self._cat_record_cnt("event_name")
        self._cat_record_cnt("name")
        self._cat_record_cnt("page")
        self._cat_record_cnt("level")
        self._cat_record_cnt("room_fqid")
        self._cat_record_cnt("fqid")
        self._cat_record_cnt("text_fqid")
        self._cat_col_nunique("text")
        self._cat_col_nunique("text_fqid")
        self._cat_col_nunique("room_fqid")
        self._cat_col_nunique("fqid")

        self._agg_features(val_cols=["room_coor_x", "room_coor_y", "screen_coor_x", "screen_coor_y"], 
                           aggs=["mean"])
        self._agg_features(val_cols=["time_diff", "hover_duration"], 
                           aggs=["mean", "max", "min", "std", "sum"])
        
        self._agg_features(val_cols=["elapsed_time", "index"], 
                           aggs=["max", "min"])

        self._cat_agg_features(val_cols=["time_diff"],
                               aggs=["mean", "max", "min", "std", "sum"],
                               cat_col="event_name")
        self._cat_agg_features(val_cols=["time_diff"],
                               aggs=["mean", "max", "min", "std", "sum"],
                               cat_col="room_fqid")
        
        self._cat_agg_features(val_cols=["time_diff"],
                               aggs=["mean", "max", "min", "std", "sum"],
                               cat_col="fqid")
        self._cat_agg_features(val_cols=["elapsed_time"],
                               aggs=["max", "min"],
                               cat_col="fqid")

        self._cat_agg_features(val_cols=["time_diff"],
                               aggs=["mean", "max", "min", "std", "sum"],
                               cat_col="text_fqid")
        self._cat_agg_features(val_cols=["elapsed_time", "index"],
                               aggs=["max", "min"],
                               cat_col="text_fqid")
        
        self._cat_agg_features(val_cols=["time_diff"],
                               aggs=["mean", "max", "min", "std", "sum"],
                               cat_col="level")
        self._cat_agg_features(val_cols=["elapsed_time", "index"],
                               aggs=["max", "min"],
                               cat_col="level")
        
        self._cat_agg_features(val_cols=["room_coor_x", "room_coor_y", "screen_coor_x", "screen_coor_y"],
                               aggs=["mean"],
                               cat_col="event_name",
                               not_use_cats=['checkpoint', 'map_hover', 'object_hover'])        
        self._cat_agg_features(val_cols=["room_coor_x", "room_coor_y", "screen_coor_x", "screen_coor_y"],
                               aggs=["mean"],
                               cat_col="name")
        
        self.result = self.result.merge(self.features, on=["session_id", "level_group"], how="left")
        return self.result

In [8]:
class FeaturesInf:
    def __init__(self, sessions_df, labels):
        self.sessions_df = sessions_df.sort_values(["elapsed_time"], ignore_index=True)
        self.result = labels
        self.group = sessions_df["level_group"].values[0]
        self.use_cols = [
            "elapsed_time", "event_name", "name", "level", "page", "index",
            "room_coor_x", "room_coor_y", "screen_coor_x", "screen_coor_y",
            "hover_duration", "text", "fqid", "room_fqid", "text_fqid"
        ]

    def _prep(self):
        # dataframeの各列をnumpy arrayで保持
        self.sessions = {}
        for c in self.use_cols:
            self.sessions[c] = self.sessions_df[c].values
        self.sessions["time_diff"] = self.sessions["elapsed_time"] - self.sessions_df["elapsed_time"].shift(1).values

    def _total_record_cnt(self):
        """level_groupごとのレコード数
        """
        add_feature = len(self.sessions["elapsed_time"])
        self.result[f"{self.group}_record_cnt"] = add_feature

    def _group_elapsed_time(self):
        """level_groupごと、epapsed_timeのmax - min（経過時間）
        """
        add_feature = np.max(self.sessions["elapsed_time"]) - np.min(self.sessions["elapsed_time"])
        self.result[f"{self.group}_group_elapsed_time"] = np.float32(add_feature)

    def _cat_record_cnt(self, cat_col):
        """level_groupごと、各{cat}のレコード数
        """
        cat_list = cat_col_lists[self.group][cat_col]
        for cat in cat_list:
            feat_name = f"{self.group}_{cat_col}_{str(cat)}_record_cnt"
            add_feature = (self.sessions[cat_col] == cat).astype(int).sum()
            self.result[feat_name] = add_feature

    def _cat_col_nunique(self, cat_col):
        """level_groupごと、[col]のユニーク数
        """
        self.result[f"{self.group}_{cat_col}_nunique"] = self.sessions_df[cat_col].dropna().nunique()       

    def _agg_features(self, val_cols, aggs):
        for val_col, agg in itertools.product(val_cols, aggs):
            feat_name = f"{self.group}_{val_col}_{agg}"
            if agg == "mean":
                add_feature = np.nanmean(self.sessions[val_col])
            elif agg == "max":
                add_feature = np.nanmax(self.sessions[val_col])
            elif agg == "min":
                add_feature = np.nanmin(self.sessions[val_col])
            elif agg == "std":
                add_feature = np.nanstd(self.sessions[val_col], ddof=1)
            elif agg == "sum":
                add_feature = np.nansum(self.sessions[val_col])
            self.result[feat_name] = np.float32(add_feature)

    def _cat_agg_features(self, val_cols, aggs, cat_col, not_use_cats=None):
        if not_use_cats is not None:
            cat_list = [c for c in cat_col_lists[self.group][cat_col] if c not in not_use_cats]
        else:
            cat_list = cat_col_lists[self.group][cat_col]

        for cat in cat_list:
            idx = self.sessions[cat_col] == cat
        
            if idx.sum() == 0:
                for val_col, agg in itertools.product(val_cols, aggs):
                    feat_name = f"{self.group}_{cat_col}_{cat}_{val_col}_{agg}"
                    self.result[feat_name] = np.float32(-1)
            else:
                for val_col, agg in itertools.product(val_cols, aggs):
                    feat_name = f"{self.group}_{cat_col}_{cat}_{val_col}_{agg}"
                    tmp = self.sessions[val_col][idx]
                    if agg == "mean":
                        add_feature = np.nanmean(tmp)
                    elif agg == "max":
                        add_feature = np.nanmax(tmp)
                    elif agg == "min":
                        add_feature = np.nanmin(tmp)
                    elif agg == "std":
                        add_feature = np.nanstd(tmp, ddof=1)
                    elif agg == "sum":
                        add_feature = np.nansum(tmp)
                    if np.isnan(add_feature):
                        self.result[feat_name] = np.float32(-1)
                    else:
                        self.result[feat_name] = np.float32(add_feature)

    def get_test(self):
        self._prep()
        self._total_record_cnt()
        self._group_elapsed_time()
        self._cat_record_cnt("event_name")
        self._cat_record_cnt("name")
        self._cat_record_cnt("page")
        self._cat_record_cnt("level")
        self._cat_record_cnt("room_fqid")
        self._cat_record_cnt("fqid")
        self._cat_record_cnt("text_fqid")
        self._cat_col_nunique("text")
        self._cat_col_nunique("text_fqid")
        self._cat_col_nunique("room_fqid")
        self._cat_col_nunique("fqid")

        self._agg_features(val_cols=["room_coor_x", "room_coor_y", "screen_coor_x", "screen_coor_y"], 
                           aggs=["mean"])
        self._agg_features(val_cols=["time_diff", "hover_duration"], 
                           aggs=["mean", "max", "min", "std", "sum"])

        self._agg_features(val_cols=["elapsed_time", "index"], 
                           aggs=["max", "min"])

        self._cat_agg_features(val_cols=["time_diff"],
                               aggs=["mean", "max", "min", "std", "sum"],
                               cat_col="event_name")
        self._cat_agg_features(val_cols=["time_diff"],
                               aggs=["mean", "max", "min", "std", "sum"],
                               cat_col="room_fqid")
        
        self._cat_agg_features(val_cols=["time_diff"],
                               aggs=["mean", "max", "min", "std", "sum"],
                               cat_col="fqid")
        self._cat_agg_features(val_cols=["elapsed_time"],
                               aggs=["max", "min"],
                               cat_col="fqid")

        self._cat_agg_features(val_cols=["time_diff"],
                               aggs=["mean", "max", "min", "std", "sum"],
                               cat_col="text_fqid")
        self._cat_agg_features(val_cols=["elapsed_time", "index"],
                               aggs=["max", "min"],
                               cat_col="text_fqid")
        
        self._cat_agg_features(val_cols=["time_diff"],
                               aggs=["mean", "max", "min", "std", "sum"],
                               cat_col="level")
        self._cat_agg_features(val_cols=["elapsed_time", "index"],
                               aggs=["max", "min"],
                               cat_col="level")
        
        self._cat_agg_features(val_cols=["room_coor_x", "room_coor_y", "screen_coor_x", "screen_coor_y"],
                               aggs=["mean"],
                               cat_col="event_name",
                               not_use_cats=['checkpoint', 'map_hover', 'object_hover'])        
        self._cat_agg_features(val_cols=["room_coor_x", "room_coor_y", "screen_coor_x", "screen_coor_y"],
                               aggs=["mean"],
                               cat_col="name")
        return self.result

In [9]:
def get_train_dataset(sessions, labels):
    # labelデータの整形
    labels = transform_labels_df_train(labels)

    # 特徴量生成
    feat = FeaturesTrain(sessions, labels)
    train = feat.get_train()
    train["question"] = train["question"].astype("category")

    return train

def get_test_dataset(sessions, labels):
    # labelデータの整形
    labels = transform_labels_df_inf(labels)

    # 特徴量生成
    feat = FeaturesInf(sessions, labels)
    test = feat.get_test()
    test["question"] = test["question"].astype("category")

    return test   

In [10]:
def calc_metrics(oof):
    logloss = log_loss(oof["correct"], oof["pred"])

    # find best th
    scores = []; thresholds = []
    best_score = 0; best_threshold = 0

    for threshold in np.arange(0.4,0.81,0.01):
        preds = (oof["pred"].values>threshold).astype(int)
        m = f1_score(oof["correct"].values, preds, average='macro')   
        scores.append(m)
        thresholds.append(threshold)
        if m>best_score:
            best_score = m
            best_threshold = threshold
    print("logloss", format(logloss, ".6f"))
    print("best_score", format(best_score, ".6f"))
    print("best_threshold", format(best_threshold, ".3f"))

    # Q別スコア
    print("---"*10)
    for q in range(18):
        q = q + 1
        preds = (oof[oof["question"]==q]["pred"].values>threshold).astype(int)
        m = f1_score(oof[oof["question"]==q]["correct"].values, preds, average='macro')
        print(f"Q{q} : F1 = {format(m, '.6f')}")
    return best_threshold

In [11]:
def run_train():
    oofs = []
    prev_features_df = None # 次のlevel_groupで特徴量を使うための保持データ。0-4は前のlevel_groupがないので初期値はNone
    dfs = []
    for group in level_group_list:
        print(group)
        # データ読み込み
        train_sessions = pd.read_csv(cfg.prep_dir + f"train{group}.csv")
        labels = pd.read_csv(cfg.prep_dir + f"train_labels{group}.csv")
        train_group = get_train_dataset(train_sessions, labels)

        # 一つ前のlevel_groupの特徴量を追加
        if prev_features_df is not None:
            train_group = train_group.merge(prev_features_df, on=["session_id"], how="left")
        else:
            pass

        # 前のlevel_groupのquestionパートの経過時間特徴量
        if group == "5-12":
            train_group["0-4_question_duration_time"] = train_group["5-12_elapsed_time_min"] - train_group["0-4_elapsed_time_max"]
            train_group["0-4_question_duration_index"] = train_group["5-12_index_min"] - train_group["0-4_index_max"]
        elif group == "13-22":
            train_group["5-12_question_duration_time"] = train_group["13-22_elapsed_time_min"] - train_group["5-12_elapsed_time_max"]
            train_group["5-12_question_duration_index"] = train_group["13-22_index_min"] - train_group["5-12_index_max"]
    
        target = "correct"
        not_use_cols = [target, "session_id", "level_group"]
        features = [c for c in train_group.columns if c not in not_use_cols]

        # 次のlevel_groupで使う用に特徴量を保持
        prev_features_df = train_group[["session_id"]+features].groupby("session_id").head(1).drop(columns="question")

        dfs.append(train_group)
    train = pd.concat(dfs, ignore_index=True)
    # concatするとcategory型がリセットされてしまうので再度cast
    train["question"] = train["question"].astype("category")

    target = "correct"
    not_use_cols = [target, "session_id", "level_group"]
    features = [c for c in train.columns if c not in not_use_cols]    

    gkf = GroupKFold(n_splits=cfg.n_splits)
    fis = []
    
    for i, (tr_idx, vl_idx) in enumerate(gkf.split(train[features], train[target], train["session_id"])):
        oof_groups = []
        print(f"fold : {i}")
        tr_x, tr_y = train.iloc[tr_idx][features], train.iloc[tr_idx][target]
        vl_x, vl_y = train.iloc[vl_idx][features], train.iloc[vl_idx][target]
        tr_data = lgb.Dataset(tr_x, label=tr_y)
        vl_data = lgb.Dataset(vl_x, label=vl_y)

        model = lgb.train(params, tr_data, valid_sets=[tr_data, vl_data],
                        num_boost_round=20000, early_stopping_rounds=100, verbose_eval=100)
        # モデル出力
        model.save_model(cfg.output_dir + f"{cfg.exp_name}/{cfg.exp_name}_model_{i}.lgb")
    
        # valid_pred
        oof_fold = train.iloc[vl_idx].copy()
        oof_fold["pred"] = model.predict(vl_x, num_iteration=model.best_iteration)
        oofs.append(oof_fold)

        # 特徴量重要度
        fi_fold = pd.DataFrame()
        fi_fold["feature"] = model.feature_name()
        fi_fold["importance"] = model.feature_importance(importance_type="gain")
        fi_fold["fold"] = i
        fis.append(fi_fold)

    fi = pd.concat(fis)    
    fi = fi.groupby("feature")["importance"].mean().reset_index()
    fi = fi.sort_values("importance", ascending=False).reset_index(drop=True)
    fi.to_csv(cfg.output_dir + f"{cfg.exp_name}/fi.csv", index=False)

    # cv
    oof = pd.concat(oofs)
    best_threshold = calc_metrics(oof)
    cfg.best_threshold = best_threshold
    oof[["session_id", "question", "pred", "correct"]].to_csv(cfg.output_dir + f"{cfg.exp_name}/oof.csv.gz", compression="gzip", index=False)

In [12]:
def get_mock_iter_train():
    """trainデータのiter分割を適用したtest_sample
    """
    test = pd.read_csv(cfg.input_dir + "_old/test.csv")
    sub = pd.read_csv(cfg.input_dir + "_old/sample_submission.csv")
    sub["level_group"] = sub["session_level"].apply(lambda x: x.split("_")[-1])
    
    # groupbyでiter作るときにgroup_levelの順番が崩れないように
    test["level_group2"] = test["level_group"].str.replace("13-22", "6")
    sub["level_group2"] = sub["level_group"].str.replace("13-22", "6")

    tests = [df[1].drop(columns=["session_level", "level_group2"]).reset_index(drop=True) for df in test.groupby("level_group2")]
    subs = [df[1].drop(columns=["session_level", "level_group2"]).reset_index(drop=True) for df in sub.groupby("level_group2")]
    return zip(tests, subs)

def get_mock_iter_test():
    """testデータのiter分割を適用したtest_sample
    """
    test = pd.read_csv(cfg.input_dir + "_old/test.csv")
    sub = pd.read_csv(cfg.input_dir + "_old/sample_submission.csv")
    
    # groupbyでiter作るときにgroup_levelの順番が崩れないように
    test["session_level"] = test["session_level"].str.replace("13-22", "6")
    sub["session_level"] = sub["session_level"].str.replace("13-22", "6")

    tests = [df[1].drop(columns="session_level").reset_index(drop=True) for df in test.groupby("session_level")]
    subs = [df[1].drop(columns="session_level").reset_index(drop=True) for df in sub.groupby("session_level")]
    return zip(tests, subs)

In [13]:
def inference(mode):
    if mode == "local_cv":
        # time series apiを模したiterをモックとして用意する
        iter_test = get_mock_iter_test()
        start_time = time.time()
    elif mode == "kaggle_inf":
        env = jo_wilder_310.make_env()
        iter_test = env.iter_test()
        
    models = []
    for i in range(cfg.n_splits):
        if mode == "local_cv":
            model_path = cfg.output_dir + f"{cfg.exp_name}/{cfg.exp_name}_model_{i}.lgb"
        elif mode == "kaggle_inf":
            model_path = f"/kaggle/input/jo-wilder-{cfg.exp_name}/{cfg.exp_name}_model_{i}.lgb"
        models.append(lgb.Booster(model_file=model_path))
    use_features = models[0].feature_name()
    
    prev_features_df = None
    for (test_sessions, sample_submission) in iter_test:
        level_group = test_sessions["level_group"].values[0]
        test = get_test_dataset(test_sessions, sample_submission)
        preds = np.zeros(len(test))

        if level_group == "0-4":
            pass
        else:
            test = test.merge(prev_features_df, on=["session_id"], how="left")

        # 前のlevel_groupのquestionパートの経過時間特徴量
        if level_group == "5-12":
            test["0-4_question_duration_time"] = test["5-12_elapsed_time_min"] - test["0-4_elapsed_time_max"]
            test["0-4_question_duration_index"] = test["5-12_index_min"] - test["0-4_index_max"]
        elif level_group == "13-22":
            test["5-12_question_duration_time"] = test["13-22_elapsed_time_min"] - test["5-12_elapsed_time_max"]
            test["5-12_question_duration_index"] = test["13-22_index_min"] - test["5-12_index_max"]

        target = "correct"
        not_use_cols = [target, "session_id", "level_group"]
        features = [c for c in test.columns if c not in not_use_cols]

        prev_features_df = test[["session_id"]+features].groupby("session_id").head(1).drop(columns="question")
        
        # そのlevel_group時点で存在しない列を追加
        complement_features = list(set(use_features) - set(test.columns.tolist()))
        test[complement_features] = np.nan

        for i in range(cfg.n_splits):
            model = models[i]
            preds += model.predict(test[use_features], num_iteration=model.best_iteration) / cfg.n_splits
        test["pred"] = preds
        preds = (preds>cfg.best_threshold).astype(int)
        sample_submission["correct"] = preds

        if mode == "local_cv":
            print(sample_submission["correct"].values)
        elif mode == "kaggle_inf":
            env.predict(sample_submission)
    if mode == "local_cv":
        process_time = format(time.time() - start_time, ".1f")
        print("sample_inf処理時間 : ", process_time, "秒")

In [14]:
prev_features_df = None # 次のlevel_groupで特徴量を使うための保持データ。0-4は前のlevel_groupがないので初期値はNone
dfs = []
for group in ["0-4", "5-12"]:
    print(group)
    # データ読み込み
    train_sessions = pd.read_csv(cfg.prep_dir + f"train{group}.csv")
    labels = pd.read_csv(cfg.prep_dir + f"train_labels{group}.csv")
    train_group = get_train_dataset(train_sessions, labels)

    # 一つ前のlevel_groupの特徴量を追加
    if prev_features_df is not None:
        train_group = train_group.merge(prev_features_df, on=["session_id"], how="left")
    else:
        pass

    # 前のlevel_groupのquestionパートの経過時間特徴量
    if group == "5-12":
        train_group["0-4_question_duration_time"] = train_group["5-12_elapsed_time_min"] - train_group["0-4_elapsed_time_max"]
        train_group["0-4_question_duration_index"] = train_group["5-12_index_min"] - train_group["0-4_index_max"]
    elif group == "13-22":
        train_group["5-12_question_duration_time"] = train_group["13-22_elapsed_time_min"] - train_group["5-12_elapsed_time_max"]
        train_group["5-12_question_duration_index"] = train_group["13-22_index_min"] - train_group["5-12_index_max"]

    target = "correct"
    not_use_cols = [target, "session_id", "level_group"]
    features = [c for c in train_group.columns if c not in not_use_cols]

    # 次のlevel_groupで使う用に特徴量を保持
    prev_features_df = train_group[["session_id"]+features].groupby("session_id").head(1).drop(columns="question")

    dfs.append(train_group)
train = pd.concat(dfs, ignore_index=True)
# concatするとcategory型がリセットされてしまうので再度cast
train_group["question"] = train["question"].astype("category")

target = "correct"
not_use_cols = [target, "session_id", "level_group"]
features = [c for c in train_group.columns if c not in not_use_cols]
gkf = GroupKFold(n_splits=cfg.n_splits)

for i, (tr_idx, vl_idx) in enumerate(gkf.split(train_group[features], train_group[target], train_group["session_id"])):
    oof_groups = []
    print(f"fold : {i}")
    tr_x, tr_y = train_group.iloc[tr_idx][features], train_group.iloc[tr_idx][target]
    vl_x, vl_y = train_group.iloc[vl_idx][features], train_group.iloc[vl_idx][target]
    tr_data = lgb.Dataset(tr_x, label=tr_y)
    vl_data = lgb.Dataset(vl_x, label=vl_y)
    break

model = optuna_lgb.train(params, tr_data, valid_sets=[tr_data, vl_data],
                         num_boost_round=20000, early_stopping_rounds=100, verbose_eval=100)

0-4
5-12
fold : 0


[I 2023-05-30 23:57:11,566] A new study created in memory with name: no-name-ffd5848f-ff56-4e9e-a063-4c503780a156
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 122655, number of negative: 65835
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 2.469526 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 472995
[LightGBM] [Info] Number of data points in the train set: 188490, number of used features: 5185
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.650724 -> initscore=0.622224
[LightGBM] [Info] Start training from score 0.622224
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.507636	valid_1's binary_logloss: 0.535095
[200]	valid_0's binary_logloss: 0.485936	valid_1's binary_logloss: 0.534311


feature_fraction, val_score: 0.534095:  14%|#4        | 1/7 [03:43<22:23, 223.93s/it]

Early stopping, best iteration is:
[153]	valid_0's binary_logloss: 0.494665	valid_1's binary_logloss: 0.534095
[LightGBM] [Info] Number of positive: 122655, number of negative: 65835
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 3.063732 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 472995
[LightGBM] [Info] Number of data points in the train set: 188490, number of used features: 5185
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.650724 -> initscore=0.622224
[LightGBM] [Info] Start training from score 0.622224
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.505179	valid_1's binary_logloss: 0.535473
[200]	valid_0's binary_logloss: 0.483392	valid_1's binary_logloss: 0.534903
[300]	valid_0's binary_logloss: 0.466536	valid_1's binary_logloss: 0.53531


...

Early stopping, best iteration is:
[128]	valid_0's binary_logloss: 0.48487	valid_1's binary_logloss: 0.533822
[LightGBM] [Info] Number of positive: 122655, number of negative: 65835
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 2.432873 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 472995
[LightGBM] [Info] Number of data points in the train set: 188490, number of used features: 5185
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.650724 -> initscore=0.622224
[LightGBM] [Info] Start training from score 0.622224
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.494142	valid_1's binary_logloss: 0.534797
[200]	valid_0's binary_logloss: 0.466068	valid_1's binary_logloss: 0.534519


min_data_in_leaf, val_score: 0.533009:  80%|########  | 4/5 [22:01<05:11, 311.58s/it]

Early stopping, best iteration is:
[137]	valid_0's binary_logloss: 0.481901	valid_1's binary_logloss: 0.534225
[LightGBM] [Info] Number of positive: 122655, number of negative: 65835
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.549104 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 472995
[LightGBM] [Info] Number of data points in the train set: 188490, number of used features: 5185
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.650724 -> initscore=0.622224
[LightGBM] [Info] Start training from score 0.622224
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.494275	valid_1's binary_logloss: 0.534388
[200]	valid_0's binary_logloss: 0.46577	valid_1's binary_logloss: 0.534298


min_data_in_leaf, val_score: 0.533009: 100%|##########| 5/5 [29:18<00:00, 351.66s/it]

Early stopping, best iteration is:
[143]	valid_0's binary_logloss: 0.48013	valid_1's binary_logloss: 0.533891


In [15]:
print(model.params)

{'objective': 'binary', 'boosting': 'gbdt', 'learning_rate': 0.1, 'metric': 'binary_logloss', 'seed': 42, 'feature_pre_filter': False, 'lambda_l1': 0.0004765648129632002, 'lambda_l2': 5.355668054611892e-06, 'num_leaves': 48, 'feature_fraction': 0.4, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 20, 'num_iterations': 20000, 'early_stopping_round': 100, 'categorical_column': [0]}


In [ ]:
model.